# Initialized

In [1]:
# At end of session
import duckdb as ddb
ddb.close()

In [2]:
import os
from dotenv import load_dotenv
import pandas as pd
import duckdb as ddb
import requests
import datetime
import time
import pandas_ta as ta

'''
from SQL.Querys import DBconn
con = ddb.connect(r'C:\Design Folder\RBGithub\Trading_Algo_Data\testingalphaoptions.duckdb')
db = DBconn(con)
'''
date = '2024-07-05'

### Get the Stock Data form Alpha Vantage

In [10]:
load_dotenv()
api_key = os.getenv("Alpha_API")
symbol = 'AAPL'
interval = '1'
Stock_lookback_period = 100
Increasing_Mins = 2

#url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=1min&month=2024-07&outputsize=full&apikey={api_key}"

stock_api = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval={interval}min&month=2024-06&entitlement=realtime&outputsize=full&apikey={api_key}'


r = requests.get(stock_api)
key = f'Time Series (1min)'
data = r.json().get(key, {})



df = pd.DataFrame.from_dict(data, orient='index')

df['1. open'] = pd.to_numeric(df['1. open'])
df['2. high'] = pd.to_numeric(df['2. high'])
df['3. low'] = pd.to_numeric(df['3. low'])
df['4. close'] = pd.to_numeric(df['4. close'])
df['5. volume'] = pd.to_numeric(df['5. volume'])

with pd.ExcelWriter('Alpha Vantage Data.xlsx') as writer:
    df.to_excel(writer, sheet_name='sheet1', index=True)

In [14]:
!pip install openpyxl


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import openpyxl

df = pd.read_excel("Alpha Vantage Data.xlsx")
print(df.info())

'''
df.index = pd.to_datetime(df.index)

#For Single Day
#stock_df = df.loc[date]

stock_df = df

new_column_names = ['open', 'high', 'low', 'close', 'volume']
stock_df.columns = new_column_names


stock_df = stock_df.copy()
stock_df['timestamp'] = stock_df.index
stock_df.reset_index(drop=True,inplace=True)

stock_df['Trade'] = False
stock_df['close'] = pd.to_numeric(stock_df['close'])
stock_df['open'] = pd.to_numeric(stock_df['open'])
clean_stock_df = stock_df

print(stock_df.info())

'''

ImportError: cannot import name 'MergedCell' from 'openpyxl.cell' (c:\Users\rybot\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\cell.py)

# Analyze Stock Data 

## EMA and Local Mins Strategy 
1. Stock_df index Starts with newest stock price
2. Swoosh
   - Price should be greater than 4
   - 1 should be greater than 3
   - 3 should be greater than 2

### Notes
1. Swoosh Points tell us When to look at the data 
2. 94 / 959 are Swoosh Points
3. Analysing Local Minmums
   1. Find all the minimums
   2. Check to see if the last X minimums are increases
   3. Check if the last X minimums are > 8 EMA
   4. Check if for the last X Minimums the 8 EMA > 20 EMA

In [20]:
stock_df['Min_Test'] = False
stock_df['EmaMin_Test'] = False
stock_df['EMA_Test'] = False
stock_df['Trade'] = False


stock_df['less1'] = stock_df['close'].shift(-1)
stock_df['less2'] = stock_df['close'].shift(-2)
stock_df['less3'] = stock_df['close'].shift(-3)
stock_df['Swoosh'] = (stock_df['less1']<stock_df['close']) & (stock_df['less2']<stock_df['less1']) & (stock_df['less2']<stock_df['less3'])

swoosh_points = stock_df[stock_df['Swoosh']]

for index, row in swoosh_points.iterrows():
    if index <= index+Stock_lookback_period: 

        table_range = stock_df.iloc[index:index+Stock_lookback_period-1].copy()

        # Stock Analysis 
        # Reverse table to calculate EMA correctly
        table_range_reversed = table_range.iloc[::-1].copy()
        table_range_reversed['EMA8'] = ta.ema(close=table_range_reversed['close'], length=8)
        table_range_reversed['EMA20'] = ta.ema(close=table_range_reversed['close'], length=20)

        # Reverse the EMA columns back to original order
        table_range['EMA8'] = table_range_reversed['EMA8'].iloc[::-1].values
        table_range['EMA20'] = table_range_reversed['EMA20'].iloc[::-1].values

        table_range['Local_Min'] = table_range['close'][(table_range['close'].shift(-1)>table_range['close'])&
                                                        (table_range['close'].shift(1)>table_range['close'])]

        local_mins = table_range[['close','Local_Min','EMA8','EMA20']].dropna()

        if len(local_mins) < 2:
            continue

        local_mins['Min_Test'] = local_mins['Local_Min'] > local_mins['Local_Min'].shift(-1)
        local_mins['EmaMin_Test'] = local_mins['Local_Min'] > local_mins['EMA8']
        local_mins['EMA_Test'] = local_mins['EMA8'] > local_mins['EMA20']
        
        Update_df = local_mins[['Min_Test','EmaMin_Test','EMA_Test']]

        stock_df.update(Update_df)
        table_range.update(Update_df)

        if all([local_mins['Min_Test'].iloc[:1].all(),
                local_mins['EmaMin_Test'].iloc[:1].all(),
                local_mins['EMA_Test'].iloc[:1].all()]):
            stock_df.at[index, 'Trade'] = True
            table_range.at[index, 'Trade'] = True


with pd.ExcelWriter('stockdata3.xlsx') as writer:
    stock_df.to_excel(writer, sheet_name='Sheet1', index=True)

        


print(stock_df[stock_df['Trade']].info())





<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 43 to 750
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   open              25 non-null     float64       
 1   high              25 non-null     object        
 2   low               25 non-null     object        
 3   close             25 non-null     float64       
 4   volume            25 non-null     object        
 5   timestamp         25 non-null     datetime64[ns]
 6   Trade             25 non-null     bool          
 7   WasRed            25 non-null     bool          
 8   Engulfing Candle  25 non-null     bool          
 9   EMA100            25 non-null     float64       
 10  Found_EMA100      25 non-null     int64         
 11  Min_Test          25 non-null     object        
 12  EmaMin_Test       25 non-null     object        
 13  EMA_Test          25 non-null     object        
 14  less1             25 non-null  

## RSI and Engulfing Candle (REC) Strategy 

In [ ]:
REC_df = clean_stock_df

Max_index = len(REC_df['timestamp'])-100

# Calculate Engulfing Candles
REC_df['WasRed'] = REC_df['close'].shift(-1) < REC_df['open'].shift(-1)
REC_df['Engulfing Candle'] = REC_df['WasRed'] & (REC_df['open'] < REC_df['close'].shift(-1)) & (REC_df['close'] > REC_df['open'].shift(-1))

REC_df_Reversed = REC_df.iloc[::-1].copy()
REC_df_Reversed['EMA100'] = ta.ema(close=REC_df_Reversed['close'], length=100)
REC_df['EMA100'] = REC_df_Reversed['EMA100'].iloc[::-1].values

REC_df['Found_EMA100'] = pd.Series(dtype=float)
REC_df['RSI'] = pd.Series(dtype=float)

Engulfing_df = REC_df[REC_df['Engulfing Candle']]

stock_df['Found_EMA100'] = pd.Series(dtype=float)
stock_df['RSI'] = pd.Series(dtype=float)

for index, row in Engulfing_df.iterrows():
    if index < Max_index:
        Engulfing_range = REC_df.iloc[index:index+100].copy()

        Engulfing_range_reversed = Engulfing_range.iloc[::-1].copy()
        Engulfing_range_reversed['EMA100'] = ta.ema(close=Engulfing_range_reversed['close'], length=100)
        Engulfing_range_reversed['RSI'] = ta.rsi(close=Engulfing_range_reversed['close'])

        Engulfing_range['EMA100'] = Engulfing_range_reversed['EMA100'].iloc[::-1].values
        Engulfing_range['RSI'] = Engulfing_range_reversed['RSI'].iloc[::-1].values

        
        REC_df['Found_EMA100'].update(Engulfing_range['EMA100'])
        REC_df['RSI'].update(Engulfing_range['RSI'])


        if Engulfing_range['close'].iloc[0] > Engulfing_range['EMA100'].iloc[0] + (Engulfing_range['EMA100'].iloc[0] * .05):
            if Engulfing_range['RSI'] > 55:
                REC_df.at[index, 'Trade'] = True

Tradable_df = REC_df[REC_df['Trade']]

print(Tradable_df.info())
        
with pd.ExcelWriter('RECdata.xlsx') as writer:
    REC_df.to_excel(writer, sheet_name='Sheet1', index=True)


# MACD Strategy

# Getting Data From Options DB 
1. Get a List of Options Log Times
2. Write a For Loop that goes through 

In [ ]:
test_df = stock_df.copy()

table_range_reversed = test_df.iloc[::-1].copy()
table_range_reversed['EMA8'] = ta.ema(close=table_range_reversed['close'], length=8)
table_range_reversed['EMA20'] = ta.ema(close=table_range_reversed['close'], length=20)

# Reverse the EMA columns back to original order
test_df['EMA8'] = table_range_reversed['EMA8'].iloc[::-1].values
test_df['EMA20'] = table_range_reversed['EMA20'].iloc[::-1].values

test_df['Local_Min'] = test_df['close'][(test_df['close'].shift(-1)>test_df['close'])&
                                                (test_df['close'].shift(1)>test_df['close'])]


with pd.ExcelWriter('Stockdf2.xlsx') as writer:
  test_df.to_excel(writer, sheet_name='Sheet1', index=True)